In [7]:
import requests
import pandas as pd
from tqdm.notebook import tqdm
from typing import Dict

In [19]:
data = pd.read_csv("../../yappy_hackaton_2024_400k.csv").iloc[40001:45000]
data.fillna("", inplace=True)

In [20]:
data.head().T.to_dict()

{40001: {'link': 'https://cdn-st.rutubelist.ru/media/0f/fc/00fc6050405ca29a97843d91b4ef/fhd.mp4',
  'description': 'Нравится ее стиль'},
 40002: {'link': 'https://cdn-st.rutubelist.ru/media/8a/04/c51a9e644f578ac328c778124d02/fhd.mp4',
  'description': 'Стесняшка Жэньши • Монолог фармацевта | Kusuriya no Hitorigoto'},
 40003: {'link': 'https://cdn-st.rutubelist.ru/media/9f/27/4edb34bc4ce182bfcd9c6822bd56/fhd.mp4',
  'description': '#спорт #наспорте #растяжка #фитнес #мотивация'},
 40004: {'link': 'https://cdn-st.rutubelist.ru/media/13/83/ed72c0e847c9964bd75061297e5a/fhd.mp4',
  'description': '#красивыедевушки #губы #макияж'},
 40005: {'link': 'https://cdn-st.rutubelist.ru/media/fb/73/16eb29534024a2cb4e9c99148cb0/fhd.mp4',
  'description': '#наспорте #тренировка #упражнения'}}

In [21]:
HOST: str = 'http://66.151.35.150:8080'
ENDPOINT_INDEX_URL: str = f'{HOST}/index'
ENDPOINT_SEARCH_URL: str = f'{HOST}/search'
ENDPOINT_SUGGEST_URL: str = f'{HOST}/suggest'
HEADERS: Dict[str, str] = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
}

In [22]:
%%time

json_data = data.iloc[1].to_dict()
response = requests.post(ENDPOINT_INDEX_URL, headers=HEADERS, json=json_data)

CPU times: user 5.93 ms, sys: 4.75 ms, total: 10.7 ms
Wall time: 12.8 s


In [23]:
response.json()

{'video_url': 'https://cdn-st.rutubelist.ru/media/8a/04/c51a9e644f578ac328c778124d02/fhd.mp4',
 'embedding': [0.0029276851564645767,
  0.013492869213223457,
  -0.012352691031992435,
  0.014194323681294918,
  0.056193701922893524,
  -0.01067755650728941,
  -0.013821701519191265,
  -0.04830105975270271,
  0.02876974642276764,
  0.011886119842529297,
  0.021669456735253334,
  -0.013052205555140972,
  0.1530025452375412,
  0.031122254207730293,
  -0.04086323827505112,
  -0.055384475737810135,
  0.010135107673704624,
  -0.02295132353901863,
  0.050698116421699524,
  -0.0036755548790097237,
  0.045795321464538574,
  -0.015235286206007004,
  0.022795211523771286,
  -0.014410442672669888,
  0.018333321437239647,
  0.004213733598589897,
  0.027907127514481544,
  0.015028494410216808,
  -0.003526723477989435,
  0.04027795046567917,
  0.0382314994931221,
  -0.04016908258199692,
  0.031226560473442078,
  0.00748378736898303,
  0.030530674383044243,
  0.06622102856636047,
  -0.01952291838824749,
  

In [24]:
%%time

params = {
    'text': 'даня милохин',
}

response = requests.get(ENDPOINT_SEARCH_URL, params=params, headers=HEADERS)

CPU times: user 5.56 ms, sys: 3.45 ms, total: 9.01 ms
Wall time: 186 ms


In [25]:
response.json()

{'ids': ['https://cdn-st.rutubelist.ru/media/f6/ea/4114706248c481eeac405a8a0bdd/fhd.mp4',
  'https://cdn-st.rutubelist.ru/media/89/26/e7eb74784195b4249ac92f1934e1/fhd.mp4',
  'https://cdn-st.rutubelist.ru/media/bf/d7/9040a3c44a6e8a73f06496d8e94b/fhd.mp4',
  'https://cdn-st.rutubelist.ru/media/65/6c/d3baabc54fa88e63824b349b266c/fhd.mp4',
  'https://cdn-st.rutubelist.ru/media/c5/a8/24a2ef194ed9ae388903c999e76a/fhd.mp4',
  'https://cdn-st.rutubelist.ru/media/ce/48/f6a35cc84ed7af1d445908b3ae15/fhd.mp4',
  'https://cdn-st.rutubelist.ru/media/fd/a2/3959a7df4474abc387580a6a636c/fhd.mp4',
  'https://cdn-st.rutubelist.ru/media/94/fe/4189d5bb4411ac4ae1fbb9c8224c/fhd.mp4',
  'https://cdn-st.rutubelist.ru/media/fc/c3/3bf5bbf7469f8ab2caba09626b1f/fhd.mp4',
  'https://cdn-st.rutubelist.ru/media/34/c4/86843bb54fb69f238e1b33d57398/fhd.mp4'],
 'scores': [261.53137,
  256.88376,
  254.74266,
  195.50679,
  193.78398,
  172.78745,
  172.78745,
  170.71727,
  154.67993,
  154.48697]}

In [27]:
responses = []

for i in tqdm(range(4,30)):
    json_data = data.iloc[i].to_dict()
    response = requests.post(ENDPOINT_INDEX_URL, headers=HEADERS, json=json_data)
    if response.status_code != 200:
        print(i, response.text)
    responses.append(response)

  0%|          | 0/26 [00:00<?, ?it/s]

In [31]:
import aiohttp, asyncio

async def post_request(session, url, headers, json_data):
    async with session.post(url, headers=headers, json=json_data) as response:
        if response.status != 200:
            print(json_data, await response.text())
        return await response.json()

responses = []
batch_size = 5

async with aiohttp.ClientSession() as session:
    for i in tqdm(range(0, 100, batch_size)):
        tasks = []
        for j in range(i, min(i + batch_size, len(data))):
            json_data = data.iloc[j].to_dict()
            tasks.append(post_request(session, ENDPOINT_INDEX_URL, HEADERS, json_data))
        responses.extend(await asyncio.gather(*tasks))